<a href="https://colab.research.google.com/github/vimalabathineni/vimalabathineni.github.io/blob/master/Sentinel_2_images_of_denmark_from_2016_to_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap
import datetime

In [2]:
# Trigger the authentication flow.
ee.Authenticate(auth_mode='notebook')

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/drive%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ybChzsbFOjXZF-uP_3ExgcvmeBZX2AEbPfE3nHN_zG0&tc=4G9Ej4Klc5aSEtfBCPlm_CY06XbUp8juvRq2237vJ9Q&cc=Jyji-EoQfl4V_y48vCGLJajPzN75v4JRQXO0EUNg3UE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AUJR-x6swS_NIGVq9tZnv3Uq7Ap8lRWfGE8CP-WUzzj-iM0tIQ_uQUII-ng

Successfully saved authorization token.


In [3]:
ee.Initialize(project='vimalasentinel2')


In [4]:
# Step 4: Define region of interest (ROI) and time range
#roi = ee.Geometry.Point([10.0, 56.0])  # Example point in Denmark
denmark_roi = ee.Geometry.Rectangle([7.9, 54.5, 15.2, 58.0])  # Covers mainland Denmark


In [5]:
# Step 5: NDVI calculation function
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')  # B8 = NIR, B4 = Red
    return image.addBands(ndvi)


In [6]:
# Step 6: Loop from Jan 2016 to June 2024
start_year = 2016
end_year = 2024
end_month = 6  # June

for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        if year == end_year and month > end_month:
            break  # Stop after June 2024

        # Safe monthly range
        start_date = f"{year}-{month:02d}-01"
        end_date = f"{year}-{month:02d}-28"

In [8]:
# Load and filter Sentinel-2 collection
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(denmark_roi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .map(add_ndvi)
 # NDVI median composite
ndvi_image = s2.select('NDVI').median().clip(denmark_roi)


In [19]:
Map = geemap.Map(center=[56.0, 10.0], zoom=6)
# NDVI visualization parameters
ndvi_vis =  {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']  # Crop stress to healthy
    }

# Add layers to map
Map.addLayer(ndvi_image, ndvi_vis, 'NDVI 2016 to 2024')
Map.addLayer(denmark_roi, {}, "ROI (Denmark)")
# Display map
Map

Map(center=[56.0, 10.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [20]:
# Export task
export_task = ee.batch.Export.image.toDrive(
            image=ndvi_image,
            description=f'NDVI_Denmark_{year}_{month:02d}',
            folder='EarthEngine_NDVI',
            fileNamePrefix=f'NDVI_Denmark_{year}_{month:02d}',
            region=denmark_roi.getInfo()['coordinates'],
            scale=10,
            crs='EPSG:4326',
            maxPixels=1e13,
            fileFormat='GeoTIFF'
        )
export_task.start()
print(f"✅ Export started for NDVI: {year}-{month:02d}")
print("✅ Export started. Go to Google Drive > EarthEngineExports to download the GeoTIFF.")


✅ Export started for NDVI: 2024-07
✅ Export started. Go to Google Drive > EarthEngineExports to download the GeoTIFF.


In [ ]:
# Step 8: Start the export task
export_task.start()
print("✅ Export started. Go to Google Drive > EarthEngineExports to download the GeoTIFF.")


✅ Export started. Go to Google Drive > EarthEngineExports to download the GeoTIFF.
